<a href="https://colab.research.google.com/github/rkap786/reading_idm/blob/main/BERTembedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import transformers
import torch

#*** pretrained_model_vectorizer.py
# Library which includes all code that involves the use of pretrained
# models for contextualization
# Functions:
#    vectorize_with_pretrained_embeddings(sentences, labels): Given a list of text examples,
#    produces embeddings of dim 768 for each example
#***


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
f= pd.read_csv("https://raw.githubusercontent.com/rkap786/reading_idm/main/Data/data_passage_ques_options_pv.csv?token=GHSAT0AAAAAACUWRVQYYN5UD5U4YGR4OQ74ZUSTQQQ")

In [8]:
f.head()

,pdfname,PassNumUnq,Passage,QNoUnq,QuestionText,OptionAddInfo,option_correct_ans,option_distractor1,option_distractor2,option_distractor3,pVal,question_correct_ans,question_dis1,question_dis2,question_dis3
0,2019-released-items-ela-g4,2019-released-items-ela-g4.1,"""No one but a man can do this,"" the business ...",2019-released-items-ela-g4.1.13,"What does the phrase ""set out"" mean as it is ...",No,began her journey,grabbed her suitcase,accepted work,started writing,0.85,"What does the phrase ""set out"" mean as it is ...","What does the phrase ""set out"" mean as it is ...","What does the phrase ""set out"" mean as it is ...","What does the phrase ""set out"" mean as it is ..."
1,2019-released-items-ela-g4,2019-released-items-ela-g4.1,"""No one but a man can do this,"" the business ...",2019-released-items-ela-g4.1.14,Read this sentence from paragraph 6.\n *Trave...,No,It shows one way to help her reach her goal.,It shows how she is the same as the character ...,It shows a young reporter exploring the world ...,It shows that a young woman taking a trip alon...,0.51,Read this sentence from paragraph 6.\n *Trave...,Read this sentence from paragraph 6.\n *Trave...,Read this sentence from paragraph 6.\n *Trave...,Read this sentence from paragraph 6.\n *Trave...
2,2019-released-items-ela-g4,2019-released-items-ela-g4.1,"""No one but a man can do this,"" the business ...",2019-released-items-ela-g4.1.15,How does the author organize the information ...,No,by listing events in the order they happened,by comparing and contrasting the places Nellie...,by showing what caused Nellie to want to take ...,by stating how the problem of traveling so far...,0.61,How does the author organize the information ...,How does the author organize the information ...,How does the author organize the information ...,How does the author organize the information ...
3,2019-released-items-ela-g4,2019-released-items-ela-g4.1,"""No one but a man can do this,"" the business ...",2019-released-items-ela-g4.1.16,"What does the word ""journalism"" mean as it is...",No,writing for newspapers,traveling for women,finding adventures,discovering opinions,0.62,"What does the word ""journalism"" mean as it is...","What does the word ""journalism"" mean as it is...","What does the word ""journalism"" mean as it is...","What does the word ""journalism"" mean as it is..."
4,2019-released-items-ela-g4,2019-released-items-ela-g4.1,"""No one but a man can do this,"" the business ...",2019-released-items-ela-g4.1.17,Which detail from the article does the map su...,No,"""She met her challenge!"" (paragraph 8)","""Start the man, and I'll start the same day."" ...","""She reached New York in 41/2 days."" (paragrap...","""She reported to her readers what she saw, tho...",0.38,Which detail from the article does the map su...,Which detail from the article does the map su...,Which detail from the article does the map su...,Which detail from the article does the map su...


In [11]:
question= f['QuestionText']
option1= f['option_correct_ans']
option2= f['question_dis1']
option3= f['question_dis2']
option4= f['question_dis3']
ques_correct_answer= f['question_correct_ans']
ques_distractor1= f['option_distractor1']
ques_distractor2= f['option_distractor2']
ques_distractor3= f['option_distractor3']

In [12]:

def vectorize_with_pretrained_embeddings(sentences):
  """
  Produces a tensor containing a BERT embedding for each sentence in the dataset or in a
  batch
  Args:
    sentences: List of sentences of length n
  Returns:
    embeddings: A 2D torch array containing embeddings for each of the n sentences (n x d)
                where d = 768
  """

  tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-cased')
  pretrained_model = transformers.BertModel.from_pretrained('bert-base-cased', output_hidden_states=False)
  pretrained_model.eval()
  embeddings = []
  for sentence in sentences:
    with_tags = "[CLS] " + sentence + " [SEP]"
    tokenized_sentence = tokenizer.tokenize(with_tags)
    # print(tokenized_sentence)
    # print(len(tokenized_sentence))
    indices_from_tokens = tokenizer.convert_tokens_to_ids(tokenized_sentence)
    segments_ids = [1] * len(indices_from_tokens)
    tokens_tensor = torch.tensor([indices_from_tokens])
    segments_tensors = torch.tensor([segments_ids])
    # print(indices_from_tokens)
    # print(tokens_tensor)
    # print(segments_tensors)
    with torch.no_grad():
      outputs = pretrained_model(tokens_tensor, segments_tensors)[0] # The output is the
      #last hidden state of the pretrained model of shape 1 x sentence_length x BERT embedding_length
      embeddings.append(torch.mean(outputs, dim = 1))# we average across the embedding length
      #dimension to produce constant sized tensors
  print(embeddings[0].shape)
  embeddings = torch.cat(embeddings, dim = 0)
  print('Shape of embeddings tensor (n x d = 768): ', embeddings.shape)
  return embeddings.cpu().detach().numpy()


In [ ]:
#Test sentences. To test, run 'python pretrained_model_vectorizer.py' at the command line
#sentences = ["The boy is running", "The dog has been barking for the whole evening"]
#To use with real data, import pretrained_model_vectorizer, then call
#'vectorize_with_pretrained_embeddings' on your list of sentences to embed
#question.embed= vectorize_with_pretrained_embeddings(question)

torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1116, 768])


In [13]:
quesoptionembed_corr= vectorize_with_pretrained_embeddings(ques_correct_answer)
quesoptionembed1= vectorize_with_pretrained_embeddings(ques_distractor1)
quesoptionembed2= vectorize_with_pretrained_embeddings(ques_distractor2)
quesoptionembed3= vectorize_with_pretrained_embeddings(ques_distractor3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])


In [14]:
quesembed= vectorize_with_pretrained_embeddings(question)
corrembed= vectorize_with_pretrained_embeddings(option1)
optionembed1= vectorize_with_pretrained_embeddings(option2)
optionembed2= vectorize_with_pretrained_embeddings(option3)
optionembed3= vectorize_with_pretrained_embeddings(option4)

torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])


In [15]:
import numpy as np
from numpy import savetxt
%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/'
df= pd.DataFrame(quesoptionembed_corr)
df.to_csv('question_embed_correct.csv')

df= pd.DataFrame(quesoptionembed1)
df.to_csv('question_embed_dis1.csv')

df= pd.DataFrame(quesoptionembed2)
df.to_csv('question_embed_dis2.csv')

df= pd.DataFrame(quesoptionembed3)
df.to_csv('question_embed_dis3.csv')


df= pd.DataFrame(quesembed)
df.to_csv('question_embed.csv')

df= pd.DataFrame(corrembed)
df.to_csv('option_embed_correct.csv')

df= pd.DataFrame(optionembed1)
df.to_csv('optionembed1.csv')

df= pd.DataFrame(optionembed2)
df.to_csv('optionembed2.csv')

df= pd.DataFrame(optionembed3)
df.to_csv('optionembed3.csv')

#np.savetxt('/Users/radhika/Downloads/question_embed.csv', question.embed, delimiter=',')
#np.save(question.embed,)


In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.330693,0.038969,0.069046,0.141347,0.562531,-0.034794,-0.045604,0.071342,-0.053162,-0.217772,...,0.317475,0.156087,-0.123962,-0.269952,0.094822,0.145907,-0.207998,0.324924,0.330233,-0.009789
1,0.150200,0.274270,-0.039747,0.272531,0.666258,-0.126865,0.024354,-0.013578,-0.016012,-0.114078,...,0.208553,0.335136,-0.002739,-0.248087,0.044126,0.209430,-0.082822,0.187699,0.172975,0.142289
2,0.316193,-0.114018,-0.007403,0.150899,0.482411,-0.054715,0.255445,0.073818,-0.062459,-0.065642,...,0.376271,0.203655,-0.218886,-0.109045,0.187939,0.178523,-0.090680,0.405097,0.209188,-0.032180
3,0.451735,-0.035361,0.060584,0.046701,0.494822,0.024858,0.058867,0.131332,-0.004860,-0.290856,...,0.245316,0.166824,-0.153981,-0.259816,-0.038695,0.186564,-0.191631,0.309455,0.253131,0.001289
4,0.375125,-0.001411,0.016060,-0.089303,0.024385,0.141236,0.317772,0.116051,-0.206724,-0.230045,...,0.256846,0.177193,-0.099984,-0.219399,0.038975,0.209707,-0.124114,0.087240,0.294826,0.216154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,0.374191,-0.092379,0.249997,0.309695,0.122369,-0.153756,0.123190,0.003419,0.053187,0.195876,...,-0.048363,0.057822,-0.082947,-0.117225,0.034189,-0.216345,-0.063492,0.308002,0.208320,-0.122181
1112,0.197396,-0.015915,0.096769,0.203303,0.251318,-0.227677,0.172645,-0.086674,-0.062594,-0.002558,...,0.036880,0.011768,0.054642,0.074262,0.104311,-0.230042,-0.028243,0.311628,-0.011404,-0.008374
1113,0.297722,-0.003906,0.007676,0.123774,0.052911,-0.238047,0.119631,0.062502,0.060466,0.146408,...,0.037549,0.198642,-0.075447,-0.001407,0.087634,-0.141926,-0.089317,0.286568,0.183502,-0.043947
1114,0.408570,0.140626,0.007848,0.179747,0.132918,-0.180473,0.132025,0.076565,-0.043467,0.182478,...,0.205288,0.061859,-0.020326,-0.103721,0.089978,-0.167842,-0.041928,0.132927,0.166269,-0.081589


In [ ]:
df

(1116, 768)

In [ ]:
question.embed

array([[ 0.33069327,  0.0389686 ,  0.06904568, ...,  0.3249241 ,
         0.33023322, -0.00978899],
       [ 0.15020001,  0.27427033, -0.03974681, ...,  0.18769884,
         0.17297511,  0.14228924],
       [ 0.31619295, -0.11401785, -0.00740315, ...,  0.40509737,
         0.20918822, -0.03217997],
       ...,
       [ 0.29772183, -0.00390649,  0.00767591, ...,  0.28656772,
         0.18350151, -0.04394712],
       [ 0.4085702 ,  0.14062564,  0.00784751, ...,  0.13292709,
         0.1662686 , -0.0815894 ],
       [ 0.359874  ,  0.0653155 , -0.03084283, ...,  0.26369065,
         0.03838476, -0.06845871]], dtype=float32)